In [1]:
import math
import os
import sys

import numpy as np

import openmm
import openmm.unit as unit
from openmm.app import *
from openmm import CustomBondForce, CustomCVForce

from emle.models import MACEEMLE
from emle.calculator import EMLECalculator

import sire as sr

distances = ((2125, 2094, 0.7), (2119, 2087, 0.3))
k = (75*unit.kilocalorie_per_mole/unit.angstrom**2).value_in_unit(unit.kilojoule_per_mole/unit.nanometer**2)
r0 = 2.5*unit.angstroms

cv = CustomBondForce("weight*r") 
cv.addPerBondParameter("weight") 

for atom1, atom2, weight in distances: 
    cv.addBond(atom1, atom2, [weight]) 

energy_expression = "k*(weighted_distance - r0)^2"
restraint_force = openmm.CustomCVForce(energy_expression)
restraint_force.addCollectiveVariable("weighted_distance", cv)
restraint_force.addGlobalParameter("k", k)
restraint_force.addGlobalParameter("r0", r0)

/home/elliot/mambaforge/envs/emle-train-sire/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  _Jd, _W3j_flat, _W3j_indices =

1

In [2]:
#Loads AbyU system
mols = sr.load([f"AbyU_OpenMM.rst", f"AbyU_OpenMM.prm7"])

# Load the topology with OpenMM too.
prm = AmberPrmtopFile(f"AbyU_OpenMM.prm7")

#Create calculator
calculator = EMLECalculator(device="cpu", mace_model="abyu_mace.model", backend="mace")

# Create an EMLEEngine bound to the calculator on the substrate.
mols, engine = sr.qm.emle(mols, mols[1], calculator)


/home/elliot/Bristol/emle-train/emle-engine/emle/models/_mace.py:212: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  source_model = _torch.load(mace_model, map_location=devic

In [3]:
# Create a QM/MM dynamics object.
d = mols.dynamics(
    timestep="1fs",
    constraint="none",
    integrator="langevin_middle",
    temperature="300K",
    qm_engine=engine,
    map={"threads": 1},
    fixed="not atoms within 20A of atomidx 3 in mol[1]"
)

In [4]:
# Get the underlying OpenMM context.
context = d._d._omm_mols

# Get the OpenMM system.
omm_system = context.getSystem()

# Store a copy of the integrator.
integrator = context.getIntegrator().__copy__()

# Add the forces to the OpenMM system.
omm_system.addForce(restraint_force)
#omm_system.addForce(harmonic_force)

# Create a new context.
new_context = openmm.Context(omm_system, integrator, context.getPlatform())

# Set force constant K for the biasing potential.
new_context.setParameter("k", k)

#Set center of biasing potential
new_context.setParameter("r0", r0)

# Set the postions of the new context to be the same as the original context.
new_context.setPositions(context.getState(getPositions=True).getPositions())



In [6]:
# Sampling production. Trajectories are saved in dcd files.
file_handle = open(f"./AbyU_OpenMM/AbyU_OpenMM.dcd", "bw")
state = new_context.getState(getPositions=True)
positions = state.getPositions()
dcd_file = DCDFile(file_handle, prm.topology, dt=integrator.getStepSize())
dcd_file.writeModel(positions)

for x in range(1000):
    integrator.step(10)
    state = new_context.getState(getPositions=True)
    positions = state.getPositions()
    dcd_file.writeModel(positions)
        
file_handle.close()

In [7]:
traj = sr.load("AbyU_OpenMM.prm7", "AbyU_OpenMM/AbyU_OpenMM.dcd")

#To view the whole system
#traj.trajectory().view()

# To view just the substrate
traj[1].trajectory().view()

NGLWidget(max_frame=100)